# Keras 高层接口

### 1. 常见网络层类

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, metrics

- 在 `tf.keras.layers` 命名空间(下文使用 `layers` 指代 `tf.keras.layers`)中提供了大量常见网络层的类
- 如全连接层、激活函数层、池化层、卷积层、循环神经网络层等。
- 对于这些网络 层类，只需要在创建时指定网络层的相关参数，并调用`__call__`方法即可完成前向计算。
- 在 调用`__call__`方法时，`Keras` 会自动调用每个层的前向传播逻辑，这些逻辑一般实现在类的 `call` 函数中。 

In [2]:
x = tf.constant([2., 1., 0.1])
layer = layers.Softmax(axis=-1)
out = layer(x)
out
# out = tf.nn.softmax(x)

<tf.Tensor: id=2, shape=(3,), dtype=float32, numpy=array([0.6590012 , 0.24243298, 0.09856589], dtype=float32)>

**Sequential**

In [3]:
network = Sequential([
    layers.Dense(3, activation=None),
    layers.ReLU(),
    layers.Dense(2, activation=None),
    layers.ReLU()
])
x = tf.random.normal([4,3]) 
out = network(x) # 输入从第一层开始，逐层传播至输出层，并返回输出层的输出 
out

<tf.Tensor: id=62, shape=(4, 2), dtype=float32, numpy=
array([[0.        , 0.        ],
       [0.        , 0.6055509 ],
       [0.        , 0.08298903],
       [0.        , 2.9622707 ]], dtype=float32)>

- `Sequential` 容器也可以通过 `add()`方法继续追加新的网络层，实现动态创建网络的功能

In [4]:
layers_num = 2
network = Sequential([])
for _ in range(layers_num):
    network.add(layers.Dense(3))
    network.add(layers.ReLU())
network.build(input_shape=(4, 4))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              multiple                  15        
_________________________________________________________________
re_lu_2 (ReLU)               multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  12        
_________________________________________________________________
re_lu_3 (ReLU)               multiple                  0         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


- 查看 `trainable_variables` 和 `variables` 参数 

In [5]:
for p in network.trainable_variables: 
    print(p.name, p.shape) # 参数名和形状

dense_2/kernel:0 (4, 3)
dense_2/bias:0 (3,)
dense_3/kernel:0 (3, 3)
dense_3/bias:0 (3,)


### 2. 模型的装配、训练与测试
**模型装配**
- `Sequential` 也是 `Model` 的子类，因此具有 `Model` 类的所有功能

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [7]:
X, y = datasets.make_moons(n_samples=200, noise=0.2, random_state=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=333)
print(X_train.shape, y_train.shape)

(150, 2) (150,)


In [8]:
# 创建 5 层的全连接网络 
network = Sequential([layers.Dense(25, activation='relu'),                      
                      layers.Dense(50, activation='relu'),                      
                      layers.Dense(50, activation='relu'),                      
                      layers.Dense(25, activation='relu'),                      
                      layers.Dense(2)]) 
network.build(input_shape=(150, 2)) 
network.summary() 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  75        
_________________________________________________________________
dense_5 (Dense)              multiple                  1300      
_________________________________________________________________
dense_6 (Dense)              multiple                  2550      
_________________________________________________________________
dense_7 (Dense)              multiple                  1275      
_________________________________________________________________
dense_8 (Dense)              multiple                  52        
Total params: 5,252
Trainable params: 5,252
Non-trainable params: 0
_________________________________________________________________


**模型装配**
- 通过 `compile` 函数指定网络使用的优化器对象、损失函数类型，评价指标等设定

In [9]:
from tensorflow.keras import optimizers, losses

In [10]:
# 模型装配
# 采用 Adam 优化器，学习率为 0.01;采用交叉熵损失函数，包含 Softmax 
network.compile(optimizer=optimizers.Adam(lr=0.01),         
                loss=losses.CategoricalCrossentropy(from_logits=True), 
                metrics=['accuracy'] # 设置测量指标为准确率 
            )

**模型训练**
- 通过 `fit()`函数送入待训练的数据集和验证用的数据集

In [11]:
# 指定训练集为 train_db，验证集为 val_db,训练 5 个 epochs，每 2 个 epoch 验证一次 
# 返回训练轨迹信息保存在 history 对象中 
history = network.fit(x=X, y=y, epochs=5, 
                      validation_split=0.2, validation_freq=2) 
# history.history # 打印训练记录

Train on 160 samples, validate on 40 samples
Epoch 1/5
160/160 [==============================] - 1s 5ms/sample - loss: 0.6988 - accuracy: 0.5063
Epoch 2/5
160/160 [==============================] - 0s 969us/sample - loss: 0.7555 - accuracy: 0.5125 - val_loss: 0.8619 - val_accuracy: 0.5250
Epoch 3/5
160/160 [==============================] - 0s 100us/sample - loss: 0.9875 - accuracy: 0.5125
Epoch 4/5
160/160 [==============================] - 0s 162us/sample - loss: 0.8478 - accuracy: 0.5437 - val_loss: 1.0462 - val_accuracy: 0.5000
Epoch 5/5
160/160 [==============================] - 0s 113us/sample - loss: 1.0342 - accuracy: 0.6438


**模型测试**
- 通过`Model.predict()`方法完成模型的预测

In [12]:
out = network.predict(X_test)
y_predict = np.argmax(out, axis=1)
y_predict.shape

(50,)

In [13]:
network.evaluate(x=X_test, y=y_test)

50/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.3236670589447022, 0.94]

### 3. 模型的保存与加载
**张量方式**
- 轻量级，仅仅保存张量参数的数值
- 需要使用**相同的网络结构**才能够正确恢复网络状态
- `Model.save_weights(path)`保存模型网络参数
- `Model.load_weights(path)`加载模型网络参数

In [30]:
# 保存模型参数到文件上 
network.save_weights('weights.ckpt') 
print('saved weights.') 
del network # 删除网络对象 

saved weights.


In [31]:
# 重新创建相同的网络结构 
network = Sequential([layers.Dense(25, activation='relu'),                      
                      layers.Dense(50, activation='relu'),                      
                      layers.Dense(50, activation='relu'),                      
                      layers.Dense(25, activation='relu'),                      
                      layers.Dense(2)]) 
network.build(input_shape=(150, 2)) 
network.compile(optimizer=optimizers.Adam(lr=0.01),         
                loss=losses.CategoricalCrossentropy(from_logits=True), 
                metrics=['accuracy'] # 设置测量指标为准确率 
            )
# 从参数文件中读取数据并写入当前网络 
network.load_weights('weights.ckpt') 
print('loaded weights!') 

loaded weights!


**网络方式**

- 保存模型参数文件
- `Model.save(path)`保存模型结构
- `keras.models.load_model(path)`恢复网络结构

In [20]:
# 保存模型结构与模型参数到文件 
network.save('model.h5') 
print('saved total model.') 
del network 
# 从文件恢复网络结构与网络参数 
network = keras.models.load_model('model.h5')  
print('loaded model!') 

saved total model.
loaded model!


**SavedModel 方式**
- `tf.saved_model.save(network, path)`
- `tf.saved_model.load(path)` 

In [32]:
# 保存模型结构与模型参数到文件 
# tf.saved_model.save(network, 'model-savedmodel') 
# print('saving savedmodel.') 
# del network # 删除网络对象 
# print('load savedmodel from file.') # 从文件恢复网络结构与网络参数 
# nn =  tf.saved_model.load('model-savedmodel')

### 4. 自定义网络

- 对于自定义的网络层，至少需要实现初始化`__init__`方法和前向传播逻辑 `call` 方法

**实现一个没有偏执向量的全链路层，激活函数为 ReLU 函数**

In [38]:
class MyDense(layers.Layer):
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()
        # 创建权值张量并添加到类管理列表中，设置为需要优化 
        self.kernel = self.add_weight('w', [inp_dim, outp_dim], trainable=True)
    
    def call(self, inputs, training=None):
        """
        training 参数用于指定模型的状态
        training 为 True 时执 行训练模式，
        training 为 False 时执行测试模式
        默认参数为 None，即测试模式
        由于全连 接层的训练模式和测试模式逻辑一致，此处不需要额外处理
        """
        # 实现自定义类的前向计算逻辑
        out = inputs @ self.kernel
        out = tf.nn.relu(out)
        return out

In [36]:
net = MyDense(4, 3) # 创建输入为 4，输出为 3 节点的自定义层
net.variables

[<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.72229   ,  0.72526014, -0.7536839 ],
        [ 0.48328388, -0.10551929, -0.27486187],
        [ 0.637228  ,  0.8125924 , -0.76559246],
        [ 0.6066965 , -0.7322022 , -0.34600782]], dtype=float32)>]

**自定义网络**

In [40]:
net = Sequential([MyDense(2, 25),
                 MyDense(25, 50),
                 MyDense(50, 50),
                 MyDense(50, 25),
                 MyDense(25, 2)])
net.build(input_shape=(None, 2))
net.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_7 (MyDense)         multiple                  50        
_________________________________________________________________
my_dense_8 (MyDense)         multiple                  1250      
_________________________________________________________________
my_dense_9 (MyDense)         multiple                  2500      
_________________________________________________________________
my_dense_10 (MyDense)        multiple                  1250      
_________________________________________________________________
my_dense_11 (MyDense)        multiple                  50        
Total params: 5,100
Trainable params: 5,100
Non-trainable params: 0
_________________________________________________________________


**自定义网络类**

In [41]:
class MyModel(keras.Model):
    # 自定义网络类，继承自 Model 基类
    def __init__(self):
        super(MyModel, self).__init__()
        # 完成网络内需要的网络层的创建工作 
        self.fc1 = MyDense(2, 25)
        self.fc2 = MyDense(25, 50)        
        self.fc3 = MyDense(50, 50)        
        self.fc4 = MyDense(50, 25)        
        self.fc5 = MyDense(25, 2)        
        
    def call(self, inputs, training=None):
        # 自定义向前运算逻辑
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.fc3(x)                
        x = self.fc4(x)                        
        x = self.fc5(x)
        return x

### 5. 测量工具

- `keras.metrics.Mean()`
- `keras.metrics.Accuracy()`
- `keras.metrics.CosineSimilarity()`
- `...`

**新建测量qi**

In [ ]:
loss_meter = metrics.Mean()

**写入数据**
- 将上述采样代码放置在每个`Batch`运算结束后，测量器会自动根据采样的数据来统计平均值

In [ ]:
# 记录采样的数据，通过 float()函数将张量转换为普通数值 
# loss_meter.update_state(float(loss)) 

**读取统计信息**

In [ ]:
# 打印统计期间的平均 loss 
# print(step, 'loss:', loss_meter.result()) 

**清除状态**

In [ ]:
# if step % 100 == 0: 
#     # 打印统计的平均 loss         
#     print(step, 'loss:', loss_meter.result())  
#     loss_meter.reset_states() # 打印完后，清零测量器 